In [80]:
import pandas as pd
train = pd.read_csv('/kaggle/input/titanic2/train.csv')
test = pd.read_csv('/kaggle/input/titanic2/test.csv')
# len(train)
# train.info()
# train.describe()
# train.isnull().mean()*100 
# test.isnull().sum()



In [81]:
#split data

y = train['Survived']
X = train.drop(columns=['Survived'])

numerics = X_train.select_dtypes(include=['int64','float64']).columns
categorical = X_train.select_dtypes(include=['object']).columns
 
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state = 0, stratify = y)


In [82]:
#fill NaN
num_medians = X_train[numerics].median()
cat_modes   = X_train[categorical].mode().iloc[0]

X_train[numerics] = X_train[numerics].fillna(num_medians)
X_test[numerics]  = X_test[numerics].fillna(num_medians)

X_train[categorical] = X_train[categorical].fillna(cat_modes)
X_test[categorical]  = X_test[categorical].fillna(cat_modes)

In [83]:
#scaler numeric 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numerics])
X_test_num = scaler.transform(X_test[numerics])

In [84]:
#onehot categorical
from sklearn.preprocessing import OneHotEncoder 

ohe = OneHotEncoder(handle_unknown = 'ignore')
X_train_cat = ohe.fit_transform(X_train[categorical])
X_test_cat = ohe.transform(X_test[categorical])

In [85]:
from scipy.sparse import hstack

X_train_final = hstack([X_train_num, X_train_cat])
X_test_final  = hstack([X_test_num, X_test_cat])


In [86]:
print(X_train_final.shape, X_test_final.shape)

(712, 1412) (179, 1412)


In [87]:
# import seaborn as sns 
# import matplotlib.pyplot as plt

# for column in train:
#     sns.boxplot(x = train[column])
#     plt.show()

In [88]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
model = KNeighborsClassifier(
    n_neighbors = 3,
    weights = 'distance',
    metric = 'euclidean'
)
param_grid = {
    'n_neighbors': range(2,12),
    'weights': ['distance', 'uniform'],
    "metric": ["euclidean" ]
}
grid_search = GridSearchCV(model, param_grid, cv = 5 )
grid_search.fit(X_train_final, y_train)



GridSearchCV(cv=5,
             estimator=KNeighborsClassifier(metric='euclidean', n_neighbors=3,
                                            weights='distance'),
             param_grid={'metric': ['euclidean'], 'n_neighbors': range(2, 12),
                         'weights': ['distance', 'uniform']})

In [89]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = grid_search.predict(X_test_final)

print(grid_search.best_score_)
print(grid_search.best_params_)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}"  )
print(classification_report(y_test, y_pred))

0.813138973702354
{'metric': 'euclidean', 'n_neighbors': 6, 'weights': 'distance'}
Accuracy: 0.79
              precision    recall  f1-score   support

           0       0.81      0.85      0.83       110
           1       0.75      0.68      0.71        69

    accuracy                           0.79       179
   macro avg       0.78      0.77      0.77       179
weighted avg       0.79      0.79      0.79       179

